# Performance Profiling Demo

## Setup


The memory_profiler python package can be run in a few ways:
* You can run the following from the command line: 
  
  `mprof run --python python my-python-script.py`
  
  This will collect function-specific data, as well as create a file with data that can be displayed as a graph using `mprof plot`
* You can see line-by-line at the command line.  First, decorate the functions you want to profile with the @profile tag, and then run
  
  `python -m memory_profiler my-python-script.py`
* Alternatively, you can use the notebook extention, with magic `%mprun`.  This accepts a -f parameter to specify the function you'd like to profile without adding the `@profile` decorator.  (example below)

### Install memory profiler
The following command will load memory_profiler into your environment.

`pip install memory_profiler`

### Register memory profiler magic commands in the notebook
The following cell will load the memory profiler notebook extension to enable profiling magics.

In [1]:
%load_ext memory_profiler

### Import library code to profile
For this demo, we'll load a few different versions of schemas.py, and load them to different module variables so we can reference them separately throughout the demo. This is only needed because we have different versions of the same code that we will be profiling.  Normally this would not be needed.

In [13]:
import importlib
import sys
spec = importlib.util.spec_from_file_location("schemas-current", "schemas-0-current.py")
schemas_current = importlib.util.module_from_spec(spec)
spec.loader.exec_module(schemas_current)

spec = importlib.util.spec_from_file_location("schemas-evaluatefhirpath", "schemas-1-evaluatefhirpath.py")
schemas_evaluatefhirpath = importlib.util.module_from_spec(spec)
spec.loader.exec_module(schemas_evaluatefhirpath)

spec = importlib.util.spec_from_file_location("schemas-uncachedpathcompile", "schemas-2-uncachedpathcompile.py")
schemas_uncachedpathcompile = importlib.util.module_from_spec(spec)
spec.loader.exec_module(schemas_uncachedpathcompile)

spec = importlib.util.spec_from_file_location("schemas-fulltableextract", "schemas-3-fulltableextract.py")
schemas_fulltableextract = importlib.util.module_from_spec(spec)
spec.loader.exec_module(schemas_fulltableextract)


## Memory Profiling

 

In [9]:
from phdi.azure import AzureFhirServerCredentialManager
from pathlib import Path
schema_path = Path("example_schema.yaml")  # Path to a schema config file.
output_path = Path("output")                    # Path to directory where tables will be written
output_format = "parquet"            # File format of tables
fhir_url = "https://pitest-fhir.azurehealthcareapis.com"           # The URL for a FHIR server
cred_manager = AzureFhirServerCredentialManager(fhir_url)

In [ ]:
!python -m memory_profiler schemas-3-fulltableextract.py example_schema.yaml output/fullextract parquet https://pitest-fhir.azurehealthcareapis.com

In [ ]:
!mprof run --output fulltableextract_mprofile.dat --python python schemas-3-fulltableextract.py example_schema.yaml output/fullextract parquet https://pitest-fhir.azurehealthcareapis.com

In [14]:
%mprun -f schemas_fulltableextract.demo_run schemas_fulltableextract.demo_run(schema_path, output_path / "fulltableextract", output_format, fhir_url)

Filename: schemas-3-fulltableextract.py

Line #    Mem usage    Increment  Occurrences   Line Contents
   214    144.6 MiB    144.6 MiB           1   def demo_run(
   215                                             schema_path: str, base_output_path: Path, output_format: str, fhir_url: str
   216                                         ):
   217    144.6 MiB      0.0 MiB           1       cred_manager = AzureFhirServerCredentialManager(fhir_url)
   218                                         
   219    144.6 MiB      0.0 MiB           1       schema = load_schema(schema_path)
   220    325.0 MiB      0.0 MiB           3       for table_name in schema.keys():
   221    258.5 MiB      0.0 MiB           2           output_path = base_output_path / table_name
   222    258.5 MiB      0.0 MiB           2           output_path.mkdir(parents=True, exist_ok=True)
   223    258.5 MiB      0.0 MiB           2           output_file_name = output_path / f"{table_name}.{output_format}"
   224      

In [9]:
%mprun -f schemas_current.make_table schemas_current.make_schema_tables(schema_path, output_path, output_format, fhir_url, cred_manager)

Filename: schemas-0-current.py

Line #    Mem usage    Increment  Occurrences   Line Contents
   109    198.0 MiB    131.4 MiB           2   def make_table(
   110                                             schema: dict,
   111                                             output_path: pathlib.Path,
   112                                             output_format: Literal["parquet"],
   113                                             fhir_url: str,
   114                                             cred_manager: AzureFhirServerCredentialManager,
   115                                         ):
   116                                             """
   117                                             Given the schema for a single table, make the table.
   118                                         
   119                                             :param schema: A schema specifying the desired values by FHIR resource type.
   120                                             :param output